<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import pandas as pd
import numpy as np

The data's in SQL; 563 million rows. That's a lot of rows.

In [3]:
path = "/Users/mark/dumps/all_so2_data.sql" # 86gb oh boy
df = pd.read_sql(path)

TypeError: read_sql() missing 1 required positional argument: 'con'